In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DATA_DIR = '../input/ear-dataset/EarVN1.0 dataset/Images'
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

In [3]:
train_val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="training",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 24151 files for training.


In [4]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="validation",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 4261 files for validation.


In [5]:
train_val_batches = tf.data.experimental.cardinality(train_val_ds)
train_val_batches

<tf.Tensor: shape=(), dtype=int64, numpy=755>

In [6]:
TOTAL_TRAIN_VAL = len(train_val_ds)
print(TOTAL_TRAIN_VAL)
train_ds = train_val_ds.take(int(8 * train_val_batches / 10))
val_ds = train_val_ds.skip(int(8 * train_val_batches / 10))
print(len(train_ds))
print(len(val_ds))

755
604
151


In [7]:
train_ds

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [8]:
base_model_EfficientNetV2S = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

82420632/82420632 [==============================] - 0s 0us/step


In [9]:
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D

model = tf.keras.Sequential()
model.add(base_model_EfficientNetV2S)
model.add(GlobalAveragePooling2D())
model.add(Dense(4096, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(164, activation = 'softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 7, 7, 1280)        20331360  
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 4096)              5246976   
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 164)               671908    
                                                                 
Total params: 26250244 (100.14 MB)
Trainable params: 260

In [11]:
from tensorflow.keras import optimizers, losses

model.compile(optimizer=optimizers.Adam(learning_rate=3e-4), loss='sparse_categorical_crossentropy', metrics='accuracy')

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(
    filepath='./best_efficientnetv2s_model_epoch={epoch:02d}_acc={val_accuracy:0.4f}.h5',
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    save_freq='epoch'
)

In [13]:
results = model.fit(train_ds, validation_data=val_ds, epochs=10, batch_size=32, callbacks=[mc])

Epoch 1/10


2024-01-19 15:12:05.355422: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


604/604 [==============================] - ETA: 0s - loss: 2.8609 - accuracy: 0.3246
Epoch 1: val_accuracy improved from -inf to 0.66307, saving model to ./best_efficientnetv2s_model_epoch=01_acc=0.6631.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


604/604 [==============================] - 278s 318ms/step - loss: 2.8609 - accuracy: 0.3246 - val_loss: 1.2328 - val_accuracy: 0.6631
Epoch 2/10
604/604 [==============================] - ETA: 0s - loss: 0.8581 - accuracy: 0.7610
Epoch 2: val_accuracy improved from 0.66307 to 0.81982, saving model to ./best_efficientnetv2s_model_epoch=02_acc=0.8198.h5
604/604 [==============================] - 189s 313ms/step - loss: 0.8581 - accuracy: 0.7610 - val_loss: 0.6757 - val_accuracy: 0.8198
Epoch 3/10
604/604 [==============================] - ETA: 0s - loss: 0.3906 - accuracy: 0.8841
Epoch 3: val_accuracy improved from 0.81982 to 0.83662, saving model to ./best_efficientnetv2s_model_epoch=03_acc=0.8366.h5
604/604 [==============================] - 188s 311ms/step - loss: 0.3906 - accuracy: 0.8841 - val_loss: 0.6377 - val_accuracy: 0.8366
Epoch 4/10
604/604 [==============================] - ETA: 0s - loss: 0.2544 - accuracy: 0.9237
Epoch 4: val_accuracy improved from 0.83662 to 0.83786, sav